<a href="https://colab.research.google.com/github/gsk224/Spatial_Portfolio/blob/main/CambodiaSochet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Discovering Illegal Cambodian Gold Mines Using Sentinal 1 Change Detection Algorythm

# Ingest

In [1]:
import ee

In [2]:
ee.Authenticate()

In [3]:
ee.Initialize(project="ee-gk-mapmatics")

In [8]:
!pip install PyGithub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 410.5/410.5 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 856.7/856.7 kB 22.8 MB/s eta 0:00:00


In [14]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [9]:
import geemap
import folium
import pandas as pd
import geopandas as gpd
import os
from github import Github

In [30]:
from geemap import chart
import calendar

In [46]:
import requests
from io import StringIO

In [52]:
!pip install ffmpeg-python

# Images

In [11]:
# Gold Mining Alerts Filtered 2022

gm22 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2022')

# Gold Mining Alerts Filtered 2023

gm23 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2023')

# Gold Mining Alerts Filtered 2023

gm24 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/Alerts2024')


In [28]:
# Change Map Filtered 2022

cm22 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2022')

# Gold Mining Alerts Filtered 2023

cm23 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2023')

# Gold Mining Alerts Filtered 2023

cm24 = ee.Image('projects/ee-gk-mapmatics/assets/cambodia/ChangeMap2024')

In [20]:
#  SET Area of Interest

aoipolygon = ee.Geometry.Polygon([
          [[105.762634, 12.952033],
           [105.762634, 13.015931],
           [105.849495, 13.015931],
           [105.849495, 12.952033],
           [105.762634, 12.952033]]
          ], None, False)

aoi = ee.FeatureCollection([aoipolygon])

# Create an empty image
empty = ee.Image().byte()

# Convert the area of study to an EE image object so we can visualize only the boundary
aoi_outline = empty.paint(featureCollection=aoi, color=1, width=2)

In [39]:
# Change Map Visualisation

jet = ['black', 'blue', 'cyan', 'yellow', 'red']
vis = {
    'min': 0,
    'max': 30,
    'palette': jet
}

In [42]:
# Images for slide map, post event
collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

imagesPost = (
    collection.filterBounds(aoi)
    .filterDate("2023-08-01", "2024-08-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 1))
)
imagesPost.size()

In [40]:
# Images for slide map, pre event

collection = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED")

imagesPre = (
    collection.filterBounds(aoi)
    .filterDate("2021-06-01", "2021-11-01")
    .filter(ee.Filter.lt("CLOUDY_PIXEL_PERCENTAGE", 5))
)
imagesPre.size()

In [47]:
# Selecting images for slide

image0322 = imagesPre
image1223 = ee.Image(imagesPost.toList(7).get(0))

In [45]:
# Visualising the Sentinel 2 images
visS2 = {
    "min": 0.0,
    "max": 3000,
    "bands": ["B4", "B3", "B2"],
}

In [48]:
# Select Mining Cocessions Layer
url = "https://data.opendevelopmentcambodia.net/geoserver/ODCambodia/ows?service=WFS&version=1.0.0&request=GetFeature&typeName=ODCambodia%3A0fdb0d9c-eb6b-4a1a-9bdd-f1e7402af1a3&outputFormat=application%2Fjson"

response = requests.get(url)
geojson_data = response.json()

In [49]:
# Filter by Clasification

datasource = [
    'Government data complete',
    'Government data partial',
    'Secondary source data',
    'Other data',
]
colors = [
    '008000',
    'CC5500',
    'FAE033',
    '87CEEB',

]
fillColor = [c + "A8" for c in colors]

In [50]:
# Convert the GeoJSON data to an Earth Engine FeatureCollection
concessions = ee.FeatureCollection(geojson_data).filter(
    ee.Filter.inList('data_class', datasource)
)
styled_concessions = geemap.ee_vector_style(concessions, column="data_class", labels=datasource, fillColor=fillColor, color='00000000')

# Sentinel 2 Maps with change data

In [21]:
# Split map showing before and after opening of mine. an 18month period
m = geemap.Map()
m.centerObject(aoi, 12)

right_layer = geemap.ee_tile_layer(image0223, visS2 , 'Dec 2023')
left_layer = geemap.ee_tile_layer(image0322, visS2 , 'Mar 2022')
m.split_map(left_layer, right_layer)

m

Map(center=[12.983981834348848, 105.80606449999864], controls=(ZoomControl(options=['position', 'zoom_in_text'…

In [56]:
# Interactive map for exploring the data, showing change maps and further filtered alert maps highlighting certain mining activity.
m = geemap.Map()
m.centerObject(aoi, 12)
m.addLayer(image1223, visS2, 'Dec 2023')

m.addLayer(aoi_outline, {'palette': 'red'}, 'Area of Study')

m.addLayer(styled_concessions, {}, 'Mining Concessions')
m.add_legend(title="Data Sources", labels=datasource, colors=colors)

m.addLayer(gm22, vis, 'Alerts Filtered 2022')
m.addLayer(gm23, vis, 'Alerts Filtered 2023')
m.addLayer(gm24, vis, 'Alerts Filtered 2024')
m.addLayer(cm22, vis, 'Change Map Filtered 2022')
m.addLayer(cm23, vis, 'Change Map Filtered 2023')
m.addLayer(cm24, vis, 'Change Map Filtered 2024')
m


Map(center=[12.983981834348848, 105.80606449999864], controls=(WidgetControl(options=['position', 'transparent…

# TimeLapse

In [48]:
# Annual change over the AOI

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2018,
    end_year=2025,
    start_date="01-01",
    end_date="12-31",
    frequency="year",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

In [55]:
# Monthly change from start2022 - end2023

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2022,
    end_year=2023,
    start_date="01-01",
    end_date="12-31",
    frequency="month",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

In [54]:
# Monthly change from start2023 - end2024

timelapse = geemap.sentinel2_timelapse(
  aoi,
    out_gif="sentinel2.gif",
    start_year=2023,
    end_year=2024,
    start_date="01-01",
    end_date="12-31",
    frequency="month",
    bands=["SWIR1", "NIR", "Red"],
    frames_per_second=1,
    title="Sentinel-2 Timelapse",
)
geemap.show_image(timelapse)

Generating URL...
Please wait ...
The GIF image has been saved to: /content/sentinel2.gif


Output()

# possibly useful later

In [36]:
# 5. Get Histogram of Band Values:
# Useful for understanding the distribution of values in a band
histogram = gm22.select('VV').reduceRegion(
    reducer=ee.Reducer.histogram(),
    geometry=aoi, # Replace with your area of interest
    scale=30,  # Adjust scale as needed
    maxPixels=1e13
).get('VV').getInfo()
print("Histogram:", histogram)

fig = chart.image_histogram(gm22.select('VV'), region=aoi, scale=30, max_buckets=100, min_bucket_width=1, max_raw=255, max_pixels=1e13)
fig

Histogram: None


TypeError: 'NoneType' object is not subscriptable

# Sentinel 1 mosaics for exploration

In [75]:
# Function to mask the SAR images acquired with an incidence angle
# lower or equal to 31 and greater or equal to 45 degrees.
def mask_angle(image):
    angle_mask = image.select('angle')
    return image.updateMask(angle_mask.gte(31).And(angle_mask.lte(45)))

# Function to get the SAR Collection.
def get_collection(dates, roi, orbit_pass):
    sar_coll_float = ee.ImageCollection('COPERNICUS/S1_GRD_FLOAT') \
        .filterBounds(roi) \
        .filterDate(dates[0], dates[1]) \
        .filter(ee.Filter.eq('orbitProperties_pass', orbit_pass))
    return sar_coll_float.map(mask_angle).select(['VV', 'VH'])

# Define variables: the period of time and the orbit pass.
list_of_dates = ['2022-03-01', '2022-08-01']
orbit_pass = 'DESCENDING'

# Apply the function to get the SAR collection.
sar_image_coll = get_collection(list_of_dates, aoi, orbit_pass)

# Print the SAR image collection.
print('SAR Image Collection:', sar_image_coll.getInfo())

SAR Image Collection: {'type': 'ImageCollection', 'bands': [], 'version': 1741711804711806, 'id': 'COPERNICUS/S1_GRD_FLOAT', 'features': [{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [27870, 21698], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 500169.82590810367, 0, -10, 1547022.4872775946]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'dimensions': [27870, 21698], 'crs': 'EPSG:32648', 'crs_transform': [10, 0, 500169.82590810367, 0, -10, 1547022.4872775946]}], 'version': 1718270110464424, 'id': 'COPERNICUS/S1_GRD_FLOAT/S1A_IW_GRDH_1SDV_20220310T224502_20220310T224527_042265_05099B_6810', 'properties': {'SNAP_Graph_Processing_Framework_GPF_vers': '8.0.3', 'SLC_Processing_facility_org': 'ESA', 'SLC_Processing_facility_country': 'France', 'GRD_Post_Processing_facility_org': 'ESA', 'transmitterReceiverPolarisation': ['VV', 'VH'], 'GRD_Post_Processing_start': 1647409057187, 'sliceNumber': 4, 'GRD

In [76]:

# Function to get dates in 'YYYY-MM-dd' format.
def get_dates(dd):
    return ee.Date(dd).format('YYYY-MM-dd')

# Function to get a SAR Mosaic clipped to the study area.
def mosaic_sar(dates1):
    dates1 = ee.Date(dates1)
    image_filt = sar_image_coll \
        .filterDate(dates1, dates1.advance(1, 'day'))
    return image_filt.mosaic() \
        .clip(aoi) \
        .set({
            'system:time_start': dates1.millis(),
            'dateYMD': dates1.format('YYYY-MM-dd')
        })

# Function to get a SAR Collection of mosaics by date.
dates_mosaic = ee.List(sar_image_coll.aggregate_array('system:time_start')) \
    .map(get_dates) \
    .distinct()

# Get a SAR List and Image Collection of mosaics by date.
get_mosaic_list = dates_mosaic.map(mosaic_sar)
get_mosaic_coll = ee.ImageCollection(get_mosaic_list)

# Print the mosaic collection.
print('get Mosaic Collection:', get_mosaic_coll.getInfo())


get Mosaic Collection: {'type': 'ImageCollection', 'bands': [], 'features': [{'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1646870400000, 'dateYMD': '2022-03-10', 'system:index': '0'}}, {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}, {'id': 'VH', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1, 0]}], 'properties': {'system:time_start': 1647907200000, 'dateYMD': '2022-03-22', 'system:index': '1'}}, {'type': 'Image', 'bands': [{'id': 'VV', 'data_type': {'type': 'PixelType', 'precision': 'float'}, 'crs': 'EPSG:4326', 'crs_transform': [1, 0, 0, 0, 1,

In [70]:
# Assuming 'get_mosaic_coll' is the Python variable for the ImageCollection
# Get a list of available dates in the ImageCollection
available_dates = get_mosaic_coll.aggregate_array('dateYMD').getInfo()
print(f"Available dates in ImageCollection: {available_dates}")

Available dates in ImageCollection: ['2022-03-06', '2022-03-11', '2022-03-18', '2022-03-23', '2022-03-30', '2022-04-04', '2022-04-11', '2022-04-16', '2022-04-23', '2022-04-28', '2022-05-05', '2022-05-10', '2022-05-17', '2022-05-22', '2022-05-29', '2022-06-03', '2022-06-10', '2022-06-15', '2022-06-22', '2022-06-27', '2022-07-04', '2022-07-09', '2022-07-16', '2022-07-21', '2022-07-28']


In [71]:
# Select the first and last available dates for visualization
# Make sure the dates exist in the available_dates list
first_date = available_dates[0]
last_date = available_dates[-1]

print(f"First available date: {first_date}")
print(f"Last available date: {last_date}")

First available date: 2022-03-06
Last available date: 2022-07-28


In [77]:

# Visualization parameters for the SAR image.
sar_vis = {
    'bands': ['VV', 'VH', 'VV'],
    'min': [-18, -23, 3],
    'max': [-4, -11, 15]
}

# Filter and display specific SAR images for given dates.
image1 = get_mosaic_coll \
    .filter(ee.Filter.eq('dateYMD', '2022-03-10')) \
    .first().log10().multiply(10.0)

image2 = get_mosaic_coll \
    .filter(ee.Filter.eq('dateYMD', '2022-07-20')) \
    .first().log10().multiply(10.0)


In [78]:
# Add Map
Map = geemap.Map()
Map.centerObject(aoi, 11)
Map.setOptions('SATELLITE')
Map.addLayer(aoi_outline, {'palette': 'red'}, 'Area of Study')
Map.addLayer(image1, sar_vis, 'Sentinel-1 | 2022-03-06')
Map.addLayer(image2, sar_vis, 'Sentinel-1 | 2022-07-28')
Map

Map(center=[12.983981834348848, 105.80606449999864], controls=(WidgetControl(options=['position', 'transparent…